In [64]:
import dotenv
import os
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from pinecone import Pinecone
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import WebBaseLoader
from langchain.schema import SystemMessage, HumanMessage

dotenv.load_dotenv(".env")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")



In [58]:
url = "https://google.serper.dev/search"

In [35]:
pc = Pinecone(api_key=PINECONE_API_KEY)


In [3]:
index_name = "medical-data"
# index_name = "medicalbot-vdb"
index = pc.Index(index_name)
index

In [ ]:
# from pinecone import ServerlessSpec

# pc.create_index(
#     name="medical-data",
#     dimension=768,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ),
#     deletion_protection="disabled",
# )

# Uploading DF

In [36]:

def upsert_data_to_pinecone(df):
    # Ensure the "tokens" column is in the correct format (list of floats)
    df["tokens"] = df["tokens"].apply(eval)  # Convert string representation of list to actual list
    print("1: ",len(df))
    # Prepare data for upserting into Pinecone
    df = df.fillna("")  # Replace NaN with an empty string
    print("1: ",len(df))
    print("Uploading data to Pinecone...")
    data_to_upsert = []
    for idx, row in df.iterrows():
        metadata = {
            "test_name": row["Test Name"],
            "source": row["Source"],
            "url": row["URL"],
            "text": row["text"]
        }
        data_to_upsert.append({"id": str(idx), "values": row["tokens"], "metadata": metadata})

    return data_to_upsert


In [37]:
def upload_In_Batches(data_to_upsert, batch_size=100):
    # Batch upload
    for i in range(0, len(data_to_upsert), batch_size):
        batch = data_to_upsert[i:i + batch_size]
        index.upsert(vectors=batch)
        print(f"Uploaded batch {i // batch_size + 1} of {len(data_to_upsert) // batch_size + 1}")
        
    print("Data upload complete!")

In [38]:
# Load your DataFrame
df = pd.read_csv("../DATA/Embedded_Files/medlinePlus_tokens_325.csv") 
df2 = pd.read_csv("../DATA/Embedded_Files/testing_com_tokens_325.csv") 
df3 = pd.read_csv("../DATA/Embedded_Files/files_tokens_325.csv")

In [39]:
len(df['tokens'][0])

16262

In [40]:
data_to_upsert = upsert_data_to_pinecone(df)
data_to_upsert2 = upsert_data_to_pinecone(df2)
data_to_upsert3 = upsert_data_to_pinecone(df3)

1:  2803
1:  2803
Uploading data to Pinecone...
1:  3449
1:  3449
Uploading data to Pinecone...
1:  1148
1:  1148
Uploading data to Pinecone...


In [41]:
print(len(data_to_upsert))
print(len(data_to_upsert2))
print(len(data_to_upsert3))

2803
3449
1148


In [42]:
merged_data_to_upsert = data_to_upsert + data_to_upsert2 + data_to_upsert3
len(merged_data_to_upsert)

7400

In [43]:
upload_In_Batches(merged_data_to_upsert)

Uploaded batch 1 of 75
Uploaded batch 2 of 75
Uploaded batch 3 of 75
Uploaded batch 4 of 75
Uploaded batch 5 of 75
Uploaded batch 6 of 75
Uploaded batch 7 of 75
Uploaded batch 8 of 75
Uploaded batch 9 of 75
Uploaded batch 10 of 75
Uploaded batch 11 of 75
Uploaded batch 12 of 75
Uploaded batch 13 of 75
Uploaded batch 14 of 75
Uploaded batch 15 of 75
Uploaded batch 16 of 75
Uploaded batch 17 of 75
Uploaded batch 18 of 75
Uploaded batch 19 of 75
Uploaded batch 20 of 75
Uploaded batch 21 of 75
Uploaded batch 22 of 75
Uploaded batch 23 of 75
Uploaded batch 24 of 75
Uploaded batch 25 of 75
Uploaded batch 26 of 75
Uploaded batch 27 of 75
Uploaded batch 28 of 75
Uploaded batch 29 of 75
Uploaded batch 30 of 75
Uploaded batch 31 of 75
Uploaded batch 32 of 75
Uploaded batch 33 of 75
Uploaded batch 34 of 75
Uploaded batch 35 of 75
Uploaded batch 36 of 75
Uploaded batch 37 of 75
Uploaded batch 38 of 75
Uploaded batch 39 of 75
Uploaded batch 40 of 75
Uploaded batch 41 of 75
Uploaded batch 42 of 75
U

# DATA RETRIEVAL AND RESPONSE GENERATION

In [4]:
import re

def remove_tags(text):
    pattern = re.compile(r'<think>.*?</think>', re.DOTALL)
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def only_p_tags(text):
    descriptions = re.findall(r"<p>(.*?)</p>", text)
    return descriptions

In [21]:

# Load embedding model
embedding_model = SentenceTransformer("sentence-transformers/msmarco-bert-base-dot-v5")  # Small and efficient
model_name = "deepseek-r1-distill-llama-70b"
# model_name = "llama-3.3-70b-versatile"
# model_name = "qwen-2.5-32b"

# Call Groq API for response
chat = ChatGroq(
api_key = GROQ_API_KEY,
model_name = model_name
)   

In [13]:

def get_unique_content_only(results):
    
    context = ''
    for search_result in results: 
        for match in search_result["matches"]:
            if match['metadata']['text'] not in context:
                context += f"- {match['metadata']['text']}\n"
    return context

def retrieve_context(description):
    results = []

    for i in range(len(description)):
        query_vector = embedding_model.encode(description[i]).tolist()
        search_results = index.query(vector=query_vector, top_k=5, include_metadata=True)
        results.append(search_results)
    return results

    
def generate_final_output(report,type,disease,generated_text,context):
    
    
    # Construct prompt
    prompt = f"""You are a expert doctor. You have to interpret the medical lab report of the patient. I have provided 
    you the lab report, the test type, the disease which the patient thinks he is suffereing from and some context which may assist you in interpreting the report.
    Interpret the report in layman understandable form in just 2 lines, not more than that and donot write anything else other than the interpretation. If you think that
    the disease he thinks he is suffering from do not match the report, you can mention that as well and recommend the possible diseases.In case the Context is not benificial,
    you can ignore it and answer from your own knowledge. Also score your answer out of 10, but just the number in the markdown form, nothing else. The answer should not be more than 2 to 3 lines.
    :
        
    Context: {context}
    Type: {type}
    Disease: {disease}
    Report: {report}
    Random Context (it can be wrong): {generated_text}
    Answer:
    """
    
    # # Call Groq API for response
    # chat = ChatGroq(
    # api_key = GROQ_API_KEY,
    # model_name = model_name
    # )   
    
    chain = chat | StrOutputParser()
    
    response = chain.invoke(prompt)
    response = remove_tags(response)
    return response


In [14]:

def generate_refined_prompt(query,type,disease):
    
    # Construct prompt
    prompt = f"""You are an expert doctor. I have provided the test type, 
    the suspected disease (if mentioned by the patient), and the lab report. Since the lab report 
    primarily consists of numerical values, retrieving relevant context from my RAG system is challenging.
    Your task is to generate a maximum of 200-300 words textual summary that best represents this report,
    incorporating key abnormalities or notable findings. Provide 2 descriptions which are inside the     
    <p>*</p> tag (don't use any other mode for separating them). This summary will be used to fetch the 
    most relevant medical context from my RAG, which, along with the report, will help the LLM provide an 
    accurate interpretation.

    For your information so that you can write the desrciption in a familiar way, the data i have scrapped 
    is from Testing.com, medlinplus.com and some books.
    Don't write anything else, other than the relavant interpretation.
    :
    
    Type: {type}
    Diesease: {disease}
    Report: {query}
        
    Answer:
    """
    
    chain = chat | StrOutputParser()
    
    response = chain.invoke(prompt)
    response = remove_tags(response)
    response = only_p_tags(response)
    return response # returns the list of 3 description

In [15]:

def discard_irrelevant_context(test_name,normal_ranges,retrieved_context,report):
    
    # Construct prompt
    prompt = f"""You are an expert doctor. Your task is to extract only the most relevant 
    information from the provided medical context and discard anything unrelated.  

### Given Information:
- **Test Type:** {test_name}  
- **Medical Report:** {report}  
- **Normal Ranges:** {normal_ranges}  
- **Retrieved Context:** {retrieved_context}  

### Instructions:
- Read the retrieved context carefully.  
- Identify the information that is directly relevant to the test {test_name} and the provided ranges (only
 consider the provided ranges if the normal ranges are not present in the test report, otherwise consider the provided ones).  
- Extract only the relevant paragraphs and discard anything that is not directly related.  
- Ensure that the extracted content provides useful insights about the given test.  

### Output Format:
Return the **filtered context** as clean paragraphs that are relevant to {test_name}. Do not include any unrelated information, just provide the paragraphs nothing else.  

"""
    
    chain = chat | StrOutputParser()
    
    response = chain.invoke(prompt)
    response = remove_tags(response)
    return response # final paragraphs

In [59]:
import json
payload = json.dumps({
  "q": "How to interpret the Dengue Hemorrhagic Fever report",
  "num": 2
})
headers = {
  'X-API-KEY': SERPER_API_KEY,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

data = json.loads(response.text)

# Extract URLs from the "organic" search results
urls = [entry["link"] for entry in data.get("organic", [])]

# Print the extracted URLs
print(urls)

['https://pmc.ncbi.nlm.nih.gov/articles/PMC88892/', 'https://www.ncbi.nlm.nih.gov/books/NBK430732/']


In [67]:
def scrape_and_extract(url):
    """Scrape the webpage content."""
    try:
        headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.text
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def clean_text(text):
    """
    Cleans extracted web content by:
    - Removing HTML tags
    - Replacing multiple spaces/newlines/tabs with a single space
    - Stripping leading/trailing whitespace
    """
    if not text:
        return ""

    # Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Remove extra spaces, newlines, and tabs
    text = re.sub(r"\s+", " ", text)

    return text.strip()


def get_interpretation(test_name):
    """Search the web, scrape results, and generate an LLM response."""
    search_query = f"How to interpret {test_name} report"

    payload = json.dumps({
        "q": search_query,
        "num": 2
    })
    headers = {
    'X-API-KEY': SERPER_API_KEY,
    'Content-Type': 'application/json'
    }
    url = "https://google.serper.dev/search"
    response = requests.request("POST", url, headers=headers, data=payload) 
    data = json.loads(response.text)
    
    # Extract URLs from the "organic" search results
    urls = [entry["link"] for entry in data.get("organic", [])]

    extracted_texts = []
    for url in urls:
            content = scrape_and_extract(url)
            if content:
                cleaned_text = clean_text(content)
                extracted_texts.append(cleaned_text)
    web_content = "\n\n".join(extracted_texts)
    print(web_content)
    messages = [
        SystemMessage(content="You are a medical expert providing the relavent content from the given one."),
        HumanMessage(content=f"""Based on the following information:\n\n{web_content}\n\nHow should I interpret the {test_name} report?
                     provide me the content from the given one that is relavent to the test and the 
                     normal ranges.""")
    ]

    response = chat(messages)
    response = remove_tags(response.content)
    return response

# Example Usage
test_name = "Dengue Hemorrhagic Fever"
interpretation = get_interpretation(test_name)
print(interpretation)


Dengue—How Best to Classify It - PMC Skip to main content An official website of the United States government Here's how you know Here's how you know Official websites use .gov A .gov website belongs to an official government organization in the United States. Secure .gov websites use HTTPS A lock ( Lock Locked padlock icon ) or https:// means you've safely connected to the .gov website. Share sensitive information only on official, secure websites. Search Log in Dashboard Publications Account settings Log out Search… Search NCBI Primary site navigation Search Logged in as: Dashboard Publications Account settings Log in Search PMC Full-Text Archive Search in PMC Advanced Search Journal List User Guide PERMALINK Copy As a library, NLM provides access to scientific literature. Inclusion in an NLM database does not imply endorsement of, or agreement with, the contents by NLM or the National Institutes of Health. Learn more: PMC Disclaimer | PMC Copyright Notice Clin Infect Dis. 2011 Sep 1

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `deepseek-r1-distill-llama-70b` in organization `org_01jf5rgz6qeavte6g94dc4958m` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 9869, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [16]:
test_name = " Dengue Hemorrhagic Fever "
disease = "dengue"

report = '''Characteristic	DF¹ (%)	DHF¹ (%)	N	p-value²
Demographics				
Age (median ± interquartile range)	26.0 ± 19.0	24.5 ± 18.0	201	0.905
Gender				
Male	99 (61.5)	30 (75.0)	201	0.077
Female	62 (38.5)	10 (25.0)		
Clinical Presentation				
Fever	159/161 (98.8)	39/39 (100.0)	200	0.647
Nausea and/or vomiting	93/161 (57.8)	27/40 (67.5)	201	0.173
Rash	44/161 (27.3)	31/40 (77.5)	201	<0.001
Abdominal Pain	6/86 (7.0)	1/11 (9.1)	97	0.582
Diarrhea	26/161 (16.2)	5/40 (12.5)	201	0.384
Myalgia	46/161 (28.6)	6/40 (15.0)	201	0.056
Headache	21/161 (13.0)	2/40 (5.0)	201	0.120
Cough	22/161 (13.7)	1/40 (2.5)	201	0.033
Hemorrhage	6/161 (3.7)	29/40 (72.5)	201	<0.001
Temperature (median)	38.00 ± 2.00	37.00 ± 1.00	177	0.014
Laboratory Findings				
Thrombocytopenia at presentation	116/151 (76.8)	36/40 (90.0)	191	0.047
Received platelet transfusion	25/90 (27.8)	13/32 (40.6)	122	0.131
Low Hemoglobin	16/137 (11.7)	8/37 (21.6)	174	0.102
Hematocrit level on admission (median)	39.50 ± 6.75	40.40 ± 8.80	167	0.607
Leukopenia	64/141 (45.4)	9/37 (24.3)	178	0.015
Neutropenia	35/133 (26.3)	16/34 (47.1)	167	0.018
Lymphocytosis	30/132 (22.7)	7/33 (21.2)	165	0.529
Monocytosis	32/122 (26.2)	19/33 (57.6)	155	0.001
Raised ALT	63/110 (57.3)	27/32 (84.4)	142	0.004
Raised AST	68/79 (86.1)	25/26 (96.2)	105	0.147'''


In [17]:
generated_text = generate_refined_prompt(report,test_name,disease) # list of 2 descriptions
print("Generated text: ",generated_text)
retrieved_content = retrieve_context(generated_text)
print("Retrieved content: ",retrieved_content)
unique_content = get_unique_content_only(retrieved_content)
print("Unique content: ",unique_content)
context = discard_irrelevant_context(test_name,None,unique_content,report)
print("Context: ",context)
response = generate_final_output(report,test_name,disease,generated_text,context)
print("Response: ",response)

Generated text:  ['The lab report highlights key differences between patients with Dengue Fever (DF) and Dengue Hemorrhagic Fever (DHF). Age and gender distributions show a higher proportion of males in the DHF group (75.0%) compared to DF (61.5%), though this difference is not statistically significant. Clinical presentation reveals significant differences, with DHF patients experiencing more severe symptoms such as rash (77.5% vs. 27.3%, p<0.001) and hemorrhage (72.5% vs. 3.7%, p<0.001). Fever is nearly universal in both groups, but cough is more common in DF (13.7% vs. 2.5%, p=0.033). Laboratory findings indicate more severe thrombocytopenia in DHF (90.0% vs. 76.8%, p=0.047) and higher rates of leukopenia (24.3% vs. 45.4%, p=0.015) and monocytosis (57.6% vs. 26.2%, p=0.001). Elevated liver enzymes, such as ALT (84.4% vs. 57.3%, p=0.004) and AST (96.2% vs. 86.1%, p=0.147), are also more pronounced in DHF, suggesting liver involvement.', 'The report underscores the progression from DF

In [ ]:
# 22.6 / 15.0 / 18.2
# 10
# 4.5
# 4.4
# 2.7

In [ ]:
# report = """Complete Blood Count (CBC) Report
# Test	Result	Reference Range	Units	Flag
# White Blood Cell Count (WBC)	7.5	4.0 - 11.0	x10³/μL	Normal
# Red Blood Cell Count (RBC)	3	4.5 - 5.9	x10⁶/μL	Abnormal
# Hemoglobin (Hgb)	15.0	13.5 - 17.5	g/dL	Normal
# Hematocrit (Hct)	45	40 - 52	%	Normal
# Mean Corpuscular Volume (MCV)	88	80 - 100	fL	Normal
# Mean Corpuscular Hemoglobin (MCH)	29	27 - 34	pg	Normal
# Mean Corpuscular Hemoglobin Concentration (MCHC)	34	32 - 36	g/dL	Normal
# Red Cell Distribution Width (RDW)	13.5	11.5 - 14.5	%	Normal
# Platelet Count	250	150 - 450	x10³/μL	Normal
# Neutrophils	35	40 - 70	%	Abnormal
# Lymphocytes	10	20 - 40	%	Abnormal
# Monocytes	6	2 - 10	%	Normal
# Eosinophils	3	1 - 4	%	Normal
# Basophils	1	0 - 2	%	Normal"""